# Databricks Health & Life Sciences
### Webinar FY25 Q4
## Prior Authorization Review with DBSQL, Variant, dbignite & redox-ehr-api

*** 

## Exploratory Analysis
***

In [0]:
DECLARE OR REPLACE VARIABLE prior_auth_bundle_uuid STRING DEFAULT "a710df7d-92b8-4e3e-99ca-b5df37808bc9";

SET VARIABLE prior_auth_bundle_uuid = :bundle_uuid;

SELECT prior_auth_bundle_uuid;

In [0]:
USE redox.hls_webinar_fy25q4;

In [0]:
SELECT 
  * 
FROM 
  fhir_bronze_variant 
WHERE 
  bundle_uuid = prior_auth_bundle_uuid;

In [0]:
%sql
SELECT 
  bundle_uuid
  ,entry.value:fullUrl::string as fullUrl
  ,entry.value:resource.resourceType::string as resourceType
  ,entry.value as resource
FROM 
  fhir_bronze_variant
  ,LATERAL variant_explode(fhir:entry) as entry
WHERE 
  bundle_uuid = prior_auth_bundle_uuid

In [0]:
-- just started writing this SQL statement. not finished yet, requires more fullurl from above.
WITH entry_exploded as (SELECT 
  bundle_uuid
  ,entry.value:fullUrl::string as fullUrl
  ,entry.value:resource.resourceType::string as resourceType
  ,entry.value as resource
FROM 
  fhir_bronze_variant
  ,LATERAL variant_explode(fhir:entry) as entry
WHERE 
  bundle_uuid = prior_auth_bundle_uuid
)
SELECT * FROM (
  SELECT
    bundle_uuid
    ,fullUrl
    ,resourceType
    ,resource
  FROM 
    entry_exploded
  )
  PIVOT (
    first(resource) FOR resourceType IN (
      'Claim'
      ,'Patient'
      ,'Practitioner'
      ,'Organization'
      ,'Coverage'
      ,'RelatedPerson'
      ,'ServiceRequest'
    )
  )

;

In [0]:
%sql
with service_requested as (
  select
    bundle_uuid
    ,code:coding[0].code::string as requested_pro_code
    ,ai_query(
      endpoint => 'databricks-meta-llama-3-3-70b-instruct'
      ,request => 'lookup HCPCS Procedure Code description for ' || code:coding[0].code::string
      ,returnType => 'STRING'
    ) AS requested_proc_code
    ,status::string as request_status
  from
    servicerequest
  where bundle_uuid = prior_auth_bundle_uuid
)
,claim_details as (
  select 
    bundle_uuid
    ,id::string as claim_id
    ,use::string as use
    ,diagnosis:[0].diagnosisCodeableConcept.coding[0].code::string as primary_diag_code
    ,diagnosis:[1].diagnosisCodeableConcept.coding[0].code::string as diag_code_2
    ,diagnosis:[2].diagnosisCodeableConcept.coding[0].code::string as diag_code_3
    ,ai_query(
      endpoint => 'databricks-meta-llama-3-3-70b-instruct'
      ,request => 'Lookup the ICD10 CM code descriptions for ' || diagnosis:[0].diagnosisCodeableConcept.coding[0].code::string || ' and ' || diagnosis:[1].diagnosisCodeableConcept.coding[0].code::string || ' and ' || diagnosis:[2].diagnosisCodeableConcept.coding[0].code::string || '. Then summarize in less than 200 characters.'
      ,returnType => 'STRING'
    ) AS diagnosis_code_summary
    ,priority:coding[0].code::string as priority
    ,type:coding[0].code::string as claim_type
  from 
    claim
  where 
    bundle_uuid = prior_auth_bundle_uuid
)
,patient_details as (
  select
    bundle_uuid
    ,id::string as patient_id
    ,name:[0].family::string as family_name
    ,name:[0].given[0]::string as given_name
  from 
    patient
  where 
    bundle_uuid = prior_auth_bundle_uuid
)
select 
  pd.*
  ,sr.* EXCEPT (bundle_uuid)
  ,cd.* EXCEPT (bundle_uuid)
from 
  patient_details pd INNER JOIN 
  service_requested sr ON pd.bundle_uuid = sr.bundle_uuid INNER JOIN
  claim_details cd ON pd.bundle_uuid = cd.bundle_uuid AND cd.bundle_uuid = sr.bundle_uuid
;